In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer

s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining input and target
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [3]:
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3352.000000,3374.000000,3389.000000
mean,49.600295,0.491740,236.849344,25.788634,76.006492
std,8.597889,0.500006,44.318612,4.066425,12.102961
min,32.000000,0.000000,107.000000,15.540000,46.000000
25%,42.000000,0.000000,206.000000,23.070000,68.000000
50%,49.000000,0.000000,234.000000,25.350000,75.000000
75%,56.000000,1.000000,263.000000,27.990000,83.000000
max,70.000000,1.000000,600.000000,56.800000,143.000000


In [4]:
X_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,836.000000,845.000000,848.000000
mean,49.523585,0.503538,236.209330,25.855408,75.369104
std,8.473274,0.500283,45.687016,4.136308,11.709564
min,34.000000,0.000000,124.000000,15.960000,44.000000
25%,42.000000,0.000000,204.000000,23.080000,68.000000
50%,49.000000,1.000000,233.000000,25.620000,75.000000
75%,56.000000,1.000000,262.000000,28.180000,82.000000
max,69.000000,1.000000,696.000000,42.000000,122.000000


In [5]:
## Defining the imputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(X_train)

## Imputing the missing values
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)

In [7]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.600295,0.491740,236.817404,25.786563,76.006195
std,8.597889,0.500006,44.070466,4.056926,12.101187
min,32.000000,0.000000,107.000000,15.540000,46.000000
25%,42.000000,0.000000,206.250000,23.072500,68.000000
50%,49.000000,0.000000,234.000000,25.350000,75.000000
75%,56.000000,1.000000,263.000000,27.980000,83.000000
max,70.000000,1.000000,600.000000,56.800000,143.000000


In [8]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.523585,0.503538,236.178066,25.853620,75.369104
std,8.473274,0.500283,45.362974,4.129085,11.709564
min,34.000000,0.000000,124.000000,15.960000,44.000000
25%,42.000000,0.000000,205.000000,23.080000,68.000000
50%,49.000000,1.000000,234.000000,25.605000,75.000000
75%,56.000000,1.000000,262.000000,28.157500,82.000000
max,69.000000,1.000000,696.000000,42.000000,122.000000


In [9]:
scaler = MinMaxScaler()

X_train_imp = pd.DataFrame(scaler.fit_transform(X_train_imp), columns = X_train_imp.columns)
X_test_imp = pd.DataFrame(scaler.fit_transform(X_test_imp), columns = X_test_imp.columns)

# Logistic Regression 

In [10]:
logit_md = LogisticRegression().fit(X_train_imp, Y_train)

## Predicting on test
logit_pred = logit_md.predict_proba(X_test_imp)[:, 1]

## Changing likelihoods to lables
logit_label = np.where(logit_pred < 0.1, 0, 1)

print(classification_report(Y_test, logit_label))

              precision    recall  f1-score   support

           0       0.95      0.34      0.51       719
           1       0.20      0.91      0.33       129

    accuracy                           0.43       848
   macro avg       0.58      0.63      0.42       848
weighted avg       0.84      0.43      0.48       848



# Random Forest

In [12]:
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

## Predicting on test
RF_pred = RF_md.predict_proba(X_test_imp)[:, 1]

## Changing likelihoods to lables
RF_label = np.where(RF_pred < 0.1, 0, 1)

print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.93      0.33      0.49       719
           1       0.19      0.85      0.31       129

    accuracy                           0.41       848
   macro avg       0.56      0.59      0.40       848
weighted avg       0.81      0.41      0.46       848



In [13]:
# Based on my results, I would use the logistic regression model to predict
# likelihood of TenYearCHD.